In [4]:
import pandas as pd

df_18 = pd.read_csv('MIGTAB019.txt',sep="|")
df_columns_18 = pd.read_csv('Campos MIGTAB019.csv',sep=";")
#df_18.drop('Unnamed: 3',axis=1,inplace= True)
df_18.columns = df_columns_18['NombreColumna'].tolist()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

C:\Users\GTC-88\AppData\Local\Temp\ipykernel_33188\1068307659.py:3: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv('MIGTAB019.txt',sep="|")


In [5]:
df_18[df_18['ro_operacion'] == 20911 ]

,ro_operacion,ro_concepto,ro_signo,ro_factor,ro_referencial,ro_signo_reajuste,ro_factor_reajuste,ro_referencial_reajuste,ro_valor,ro_porcentaje_efa,ro_garantia,ro_financiado
47729,20911,CAP,NaN,0.0,NaN,NaN,0.0,NaN,2002000.0,0,0,NaN
47730,20911,INT,NaN,0.0,T_FIJA,NaN,0.0,16.8,0.0,470,0,NaN
47731,20911,IMO,NaN,0.0,T_FIJA,NaN,0.0,NaN,0.0,470,0,NaN


In [7]:
df_columnasObligatorias = df_columns_18[df_columns_18['Obligatorio'] == 'S']['NombreColumna'].tolist()
df_Final = df_18[df_columnasObligatorias]
#df_Final.shape
nulos = df_Final.isnull().sum().reset_index()
nulos.columns = ['NombreColumna','CantidadNulos']
df_Columnas = nulos[nulos['CantidadNulos']> 0 ]['NombreColumna'].head(4).tolist()
